In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/in-star-craft-2-player/TRAIN_LONG.CSV.GZ
/kaggle/input/in-star-craft-2-player/SAMPLE_SUBMISSION.CSV
/kaggle/input/in-star-craft-2-player/TRAIN.CSV.GZ
/kaggle/input/in-star-craft-2-player/TEST_LONG.CSV.GZ
/kaggle/input/in-star-craft-2-player/TEST.CSV.GZ


In [15]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
import gzip
from pathlib import Path
import os
from typing import Optional
import io


In [18]:
def read_ds_gzip(path: Optional[Path]=None, ds: str = "TRAIN") -> pd.DataFrame:
    """Read and parse the gzipped CSV dataset with single pass."""
    with gzip.open(path if path else f'{ds}.CSV.gz') as f:
        # Single pass to find max columns and read data
        lines = [line.decode().strip() for line in f]
        max_actions = max(len(line.split(",")) for line in lines)
        
        # Reset file pointer using BytesIO
        file_like = io.BytesIO(b'\n'.join([line.encode() for line in lines]))
        
        _names = ["battleneturl", "played_race"] if "TRAIN" in ds else ["played_race"]
        _names.extend(range(max_actions - len(_names)))
        
        return pd.read_csv(file_like, names=_names, dtype=str)

In [19]:

# First create and fit the action encoder globally
action_encoder = LabelEncoder()

# Load training data
df_train = read_ds_gzip(Path('/kaggle/input/in-star-craft-2-player/TRAIN.CSV.GZ'))

In [30]:
# Modified format_data function with proper padding and encoding
def format_data(df):
    # 1. Encode Target
    user_encoder = LabelEncoder()
    df['user_id'] = user_encoder.fit_transform(df['battleneturl'])
    
    # 2. Process Race
    race_encoder = LabelEncoder()
    df['race_encoded'] = race_encoder.fit_transform(df['played_race'])
    
    # 3. Process Action Sequence
    action_columns = df.columns[2:-2]
    
    # Build action vocabulary with 'no_action' first
    all_actions = []
    for _, row in df.iterrows():
        for col in action_columns:
            action = row[col]
            if pd.isna(action):
                all_actions.append('no_action')
            elif action.startswith('t'):
                all_actions.append('time_marker')
            else:
                all_actions.append(action)
    
    # Force 'no_action' to be first class
    unique_actions = np.unique(all_actions)
    unique_actions = sorted(unique_actions, key=lambda x: x != 'no_action')
    action_encoder.fit(unique_actions)
    
    # Convert to sequences with right-padding
    max_length = len(action_columns)
    sequences = []
    for _, row in df.iterrows():
        seq = []
        for col in action_columns:
            action = row[col]
            if pd.isna(action):
                seq.append('no_action')
            elif action.startswith('t'):
                seq.append('time_marker')
            else:
                seq.append(action)
        # Convert to encoded values
        encoded = action_encoder.transform(seq)
        sequences.append(encoded)
    
    # Convert to padded numpy array
    sequences = np.array(sequences)
    
    return {
        'user_ids': df['user_id'].values,
        'races': df['race_encoded'].values,
        'sequences': sequences,
        'encoders': {  # Store all encoders in a sub-dictionary
            'user': user_encoder,
            'race': race_encoder,
            'action': action_encoder
        }
    }

In [39]:
import tensorflow as tf
from tensorflow.keras import layers

class MaskedMaxPooling(layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        # By setting this flag, Keras knows this layer can handle masks.
        self.supports_masking = True

    def call(self, inputs, mask=None):
        # inputs: The actual tensor you want to pool
        # mask:   A boolean mask of the same batch/time dimension shape
        x = inputs
        
        # If no mask is passed (possible if nothing upstream is masked),
        # just do a regular global max:
        if mask is None:
            return tf.reduce_max(x, axis=1)

        # Expand mask to match the shape of x for element-wise operations
        mask = tf.cast(mask, x.dtype)            # shape: (batch, timesteps)
        mask = tf.expand_dims(mask, axis=-1)     # shape: (batch, timesteps, 1)

        # Set masked elements to a very negative value so they don't affect max
        x_masked = x * mask + (1. - mask) * -1e9
        return tf.reduce_max(x_masked, axis=1)

    # Typically, after pooling, we no longer need a mask (because we've collapsed
    # the time dimension), so we return None. If you do need to pass a new mask
    # downstream, implement `compute_mask` accordingly.
    def compute_mask(self, inputs, mask=None):
        return None

In [40]:
from tensorflow.keras import backend as K

def build_model(n_users, n_races):
    # Input 1: Action sequences
    action_input = layers.Input(shape=(None,), name='action_input')
    
    # Embedding with masking
    action_emb = layers.Embedding(
        input_dim=len(formatted['encoders']['action'].classes_),
        output_dim=64,
        mask_zero=True  # Mask 0 values (padding)
    )(action_input)
    
    # Bidirectional LSTM with mask propagation
    lstm_out = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(action_emb)
    
    # Custom masked max pooling
    def masked_max_pooling(args):
        x, mask = args
        mask = K.expand_dims(K.cast(mask, K.floatx()), -1)
        x_masked = x * mask
        x_masked += (1 - mask) * -1e9  # Large negative for masked positions
        return K.max(x_masked, axis=1)
    
    # Create mask from action input
    mask = layers.Lambda(lambda x: K.not_equal(x, 0))(action_input)
    max_pool = MaskedMaxPooling()(lstm_out)
    
    # Input 2: Race
    race_input = layers.Input(shape=(1,), name='race_input')
    race_emb = layers.Embedding(n_races, 8)(race_input)
    race_flat = layers.Flatten()(race_emb)
    
    # Combine features
    combined = layers.concatenate([max_pool, race_flat])
    
    # Output
    output = layers.Dense(n_users, activation='softmax')(combined)
    
    return Model(
        inputs=[action_input, race_input],
        outputs=output
    )



In [31]:
# Usage
formatted = format_data(df_train)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'global_max_pooling1d_1' (of type GlobalMaxPooling1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


In [41]:
model = build_model(
    n_users=len(formatted['encoders']['user'].classes_),
    n_races=len(formatted['encoders']['race'].classes_)
)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [42]:
# Modified training section
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

# Convert all data to numpy arrays first
sequences = formatted['sequences']
races = formatted['races']  # Convert pandas Series to numpy array
user_ids = formatted['user_ids'] # Convert pandas Series to numpy array

# Split data properly
(train_seq, test_seq, 
 train_race, test_race,
 train_y, test_y) = train_test_split(
    sequences,
    races,
    user_ids,
    test_size=0.2,
    stratify=user_ids,
    random_state=42
)

# Calculate class weights correctly
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(train_y),
    y=train_y
)
class_weight_dict = {cls: weight for cls, weight in zip(np.unique(train_y), class_weights)}

# Verify input shapes
print(f"Action sequences shape: {train_seq.shape}")
print(f"Race features shape: {train_race.shape}")
print(f"Target labels shape: {train_y.shape}")

# Train with validation data
history = model.fit(
    x=[train_seq, train_race],
    y=train_y,
    epochs=100,
    batch_size=128,
    validation_data=([test_seq, test_race], test_y),
    class_weight=class_weight_dict,
    callbacks=[checkpoint, early_stop]
)

Action sequences shape: (2441, 10537)
Race features shape: (2441,)
Target labels shape: (2441,)
Epoch 1/100


InvalidArgumentError: Graph execution error:

Detected at node functional_4_1/bidirectional_4_1/forward_lstm_4_1/Assert/Assert defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, i